<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%95%99%EC%83%81%EB%B8%94(%ED%8A%B8%EB%A6%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

앙상블 학습

- 통상 정형데이터를 다루는 데 가장 뛰어난 성과를 냄


비정형 데이터는 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 까다롭다

**랜덤포레스트**

결정 트리를 랜덤하게 만들고 각 결정트리의 예측을 사용해 최종 예측

입력된 훈련 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듦(샘플 중복 추출 가능)

추출된 샘플 = 부트스트랩 샘플

부트스트랩 샘플을 훈련세트의 크기와 같게 생성

분류
- 전체 특성 개수의 제곱근만큼 특성을 사용
- 각 트리의 클래스 별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측

회귀
- 경우 전체 특성 사용
- 단순히 각 ㅌ트리의 에측을 평균

랜덤하게 선택한 샘플과 특성을 사용하기 때문에 과적합을 방지

검증/테스트 세트에서 안정적인 성능을 얻을 수 있음

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target,test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier # 기본적으로 100개의 트리 사용
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1) #return_train_score = True -> 훈련세트 점수도 같이 반환
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
# DecisionTreeClassifier 매개변수
# criterion, max_depth, max_feature, min_samples_split, min_impurity_decrease, min_samples_leaf

In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) # 특성 중요도

[0.23167441 0.50039841 0.26792718]


하나의 특성에 과도한 집중을 막고 좀 더 많은특성이 훈련에 기여할 수 있게함

-> 과대적합을 막고 일반화 성능을 향상

**랜덤포레스트 자체 모델 평가 기능**

OOB(out of bag) = 부트스트랩 제외 샘플
- 검증 세트 역할 수행

In [5]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42) #oob_score = -> 기본값 false
rf.fit(train_input, train_target)
print(rf.oob_score_) #oob 점수 출력

0.8934000384837406


**에스트라 트리**

부트스트랩 샘플을 사용하지 않음

결정트리 생성 시 전체 훈련세트 사용

대신 노드를 분할할 때 가장 좋은 분할이 아닌 랜덤 분할 시행

계산 속도가 빠름

In [8]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [11]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


**그레디언트 부스팅**

깊이가 얕은 결정 트리를 사용해 이전 트리의 오차를 보완

기본적으로 깊이가 3인 결정트리 100개 사용

깊이가 얕은 트리 사용으로 과대적합에 강하고 통상 높은 일반화 성능을 기대할 수 있음

학습률 매개변수로 속도 조절

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


학습률을 증가시키고, 트리 개수를 늘리면 향상될 수 있음

In [14]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42) # n_estimators -> 트리개수 / learning_rate 기본값 0.1
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [15]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


일반적으로 부스팅이 랜덤포레스트보다 조금 더 높은 성능을 얻을 수 있다

반면 부스팅의 경우 순서대로 트리를 추가하기 때문에 훈련 속도가 느림

**히스토그램 기반 그레디언트 부스팅**

입력 특성을256개 구간으로 나눔 -> 노드를 분할할 때 최적의 분할을 매우 빠르게 찾음

성능개선 관련 n_estimators 매개변수 대신, max_iter 매개변수 사용

In [16]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


**permutation_importances**

 특성을 하나씩 랜덤하게 섞어 모델 성능이 변화하는지 관찰해 어떤 특성이 중요한지 계산

 사이킷런 제공 모든 추정기 모델에 사용 가능

In [19]:
from sklearn.inspection import permutation_importance #permutation = 순열

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1) # n_repeats -> 매개변수를 섞는 횟수, 기본값 5
print(result.importances_mean) # 중요도 평균 // importances = 중요도 // importances_std =표준편차

[0.08876275 0.23438522 0.08027708]


In [21]:
hgb.score(test_input, test_target)

0.8723076923076923

xgboost -> 다양한 부스팅 알고리즘 지원

In [22]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42) # tree_method = hist -> 히스토그램기반
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


히스토그램 기반 그래디언트 부스팅 라이브러리는 ms사에서 만든 LightGBM

In [23]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
